In [ ]:
import time
import requests
from transformers import BartTokenizer, BartForConditionalGeneration
import json
import pprint

In [ ]:
# from secrets import API_KEY_LISTENNOTES, API_KEY_ASSEMBLYAI
API_KEY_LISTENNOTES = 'Enter Your Api Key'    #this is the api key  of my account listennotes.com
API_KEY_ASSEMBLYAI = 'Enter your Api Key'        # this is the api key of my account assembly ai

Listennotees_episode_endpoint = 'https://listen-api.listennotes.com/api/v2/episodes'
headers_listennotes = {
    'X-ListenAPI-Key': API_KEY_LISTENNOTES
}

In [ ]:
transcript_endpoint = 'https://api.assemblyai.com/v2/transcript'

In [ ]:
headers_assemblyai = {
    "authorization": API_KEY_ASSEMBLYAI,
    "content-type": "application/json"
}

In [ ]:
def get_episode_audio_url(episode_id):
  url = Listennotees_episode_endpoint + '/' + episode_id
  response = requests.request('GET', url, headers=headers_listennotes)
  data = response.json()
  # pprint.pprint(data)

  if 'podcast' in data:
    podcast_title = data["podcast"]['title']
  else:
    podcast_title = None

  if 'title' in data:
    episode_title = data['title']
  else:
    episode_title = None

  if 'thumbnail' in data:
    thumbnail = data['thumbnail']
  else:
    thumbnail = None

  if 'audio' in data:
    audio = data['audio']
  else:
    audio = None

  return audio, thumbnail, podcast_title, episode_title

In [ ]:
def transcribe(audio_url, auto_chapters=False):                   # here we generate the transrcibe of the podcast
    transcript_request = {
      'audio_url': audio_url,
      'auto_chapters': 'True' if auto_chapters else 'False'
    }

    transcript_response = requests.post(transcript_endpoint, json=transcript_request, headers=headers_assemblyai)
    pprint.pprint(transcript_response.json())
    return transcript_response.json()['id']

In [ ]:
#lets put here some optional keyword arguments in poll
def poll(transcript_id, **kwargs):
  polling_endpoint = transcript_endpoint + '/' + transcript_id
  polling_response = requests.get(polling_endpoint, headers=headers_assemblyai)

  if polling_response.json()['status'] == 'completed':
    filename = transcript_id + '.txt'                   # this file is use for transcription
    with open('transcript.txt', 'w') as f:
      f.write(polling_response.json()['text'])

    filename = transcript_id + '_chapters.json'           #in our json file we want to add more information
    with open('transcript_chapters.json', 'w') as f:                           #this for the auto chapter
      chapters = polling_response.json()['chapters']

      #create some dictionary
      data = {'chapters': chapters}
      for key , value in kwargs.items():
        data[key] = value

      json.dump(data, f, indent=4)

    print('Transcript saved')
    return  True
  return False

In [ ]:
def pipeline(episode_id):
  audio, thumbnail, podcast_title, episode_title = get_episode_audio_url(episode_id)    #here is also present the episode id which give podcast details
  # print(audio, thumbnail, podcast_title, episode_title)
  transcribe_id = transcribe(audio, auto_chapters=True)

  while True:
    result = poll(transcribe_id, audio=audio, thumbnail=thumbnail, podcast_title=podcast_title,episode_title=episode_title)
    if result:
      break
    print("waiting for 60 sec")
    time.sleep(60)

In [ ]:
if __name__ == "__main__":
  # audio, thumbnail, podcast_title, episode_title = get_episode_audio_url("20bf194224434ebbba8462ea8136cc3d")    #here is also present the episode id which give podcast details
  # # print(audio, thumbnail, podcast_title, episode_title)
  # transcribe_id = transcribe(audio, auto_chapters=True)
  # result = poll(transcribe_id, audio=audio, thumbnail=thumbnail, podcast_title=podcast_title, episode_title=episode_title)
  pipeline("20bf194224434ebbba8462ea8136cc3d")

{'acoustic_model': 'assemblyai_default',
 'audio_duration': None,
 'audio_end_at': None,
 'audio_start_from': None,
 'audio_url': 'https://audio.listennotes.com/e/p/20bf194224434ebbba8462ea8136cc3d/',
 'auto_chapters': 'True',
 'auto_highlights': False,
 'auto_highlights_result': None,
 'boost_param': None,
 'chapters': None,
 'confidence': None,
 'content_safety': False,
 'content_safety_labels': {},
 'custom_spelling': None,
 'custom_topics': False,
 'custom_topics_results': None,
 'disfluencies': False,
 'dual_channel': None,
 'entities': None,
 'entity_detection': False,
 'filter_profanity': False,
 'format_text': True,
 'iab_categories': False,
 'iab_categories_result': {},
 'id': 'c765e061-e1bf-4b53-b7a9-83e513d3492f',
 'is_deleted': None,
 'language_code': 'en_us',
 'language_confidence': None,
 'language_confidence_threshold': None,
 'language_detection': False,
 'language_model': 'assemblyai_default',
 'multichannel': False,
 'punctuate': True,
 'redact_pii': False,
 'redact_p

In [ ]:
# Step 2: Import the required libraries
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, pipeline
import textwrap

In [ ]:
# Step 3: Load pre-trained BART model and tokenizer for summarization
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')


In [ ]:
# Load pre-trained BERT model for summarization
bert_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


In [ ]:
# Function to read the transcribe.txt file
def read_transcript(file_path='transcript.txt'):
    with open(file_path, 'r') as f:
        transcript_text = f.read()
    return transcript_text


In [ ]:
# Function to save the summary to a text file
def save_summary_to_file(summary, model_name):
    with open(f"{model_name}_summary.txt", 'w') as f:
        f.write(summary)


In [ ]:
# Step 4: Function to summarize text using BART
def generate_bart_summary(text):
    inputs = bart_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    bart_summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return bart_summary


In [ ]:
# Step 5: Function to summarize text using BERT (chunked)
def generate_bert_summary(text):
    summary_chunks = []
    for chunk in chunk_text(text):
        summary = bert_summarizer(chunk, max_length=150, min_length=40, do_sample=False)
        summary_chunks.append(summary[0]['summary_text'])
    return " ".join(summary_chunks)  # Combine all chunk summaries

In [ ]:
# Step 6: Function to chunk large texts
def chunk_text(text, max_length=1000):
    # Split the text into manageable chunks
    return textwrap.wrap(text, width=max_length)


In [ ]:
if __name__ == "__main__":
    # Read the transcript file
    transcript_text = read_transcript()  # Ensure transcribe.txt is in the same directory

    # Chunk the transcript for summarization
    text_chunks = chunk_text(transcript_text)

    # Generate and save summaries using BART (abstractive)
    bart_full_summary = ""
    for chunk in text_chunks:
        bart_summary = generate_bart_summary(chunk)
        bart_full_summary += bart_summary + " "  # Combine the summaries

    print("\nAbstractive BART Summary:\n", bart_full_summary)
    save_summary_to_file(bart_full_summary, "bart")

    # Generate and save extractive summaries using BERT
    bert_full_summary = generate_bert_summary(transcript_text)  # Process the entire text in chunks
    print("\nExtractive BERT Summary:\n", bert_full_summary)
    save_summary_to_file(bert_full_summary, "bert")


Abstractive BART Summary:
 Indie Hackers is a podcast from Courtland from indiehackers.com. Courtland: I'm doing an episode on cryptocurrencies, specifically nfts. "I am convinced that there's a possibility that nfts could be the future of the web" This episode is for entertainment purposes only. Nothing contained in this episode is financial advice. Do not take what we're saying as advice to go out and buy a bunch of crypto. Do your own research. "If I were to tell anyone how to make money like level one, buy the coins. Just buy like $100 a week," he says. "Defi is a very lucrative thing. And nfts are very lucrative" Start with level one. Get a Coinbase account, the proper regular Coinbase account. You want to do it with Robinhood, do it. They give you the features to do that too. Level two is just a very hard gate. "I have a couple friends now in the space one of them is 23 years old. He's a systems engineer in Utah, and he made half a million dollars this year trading nfts" "You ca

Your max_length is set to 150, but your input_length is only 83. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)



Extractive BERT Summary:
 Indie Hackers is a podcast from Courtland from indiehackers.com. Courtland: I'm doing an episode on cryptocurrencies, specifically nfts. "I am convinced that there's a possibility that nfts could be the future of the web" Nothing contained in this episode is financial advice. Do not take what we're saying as advice to go out and buy a bunch of crypto. This episode is for entertainment purposes only. Enjoy. If I were to tell anyone how to make money like level one, buy the coins. Do some research, figure out the ones you want to buy. If you just want a high level, buy bitcoin, ethereum, buy like $100 a week, whatever you can afford. Get a Coinbase account, the proper regular Coinbase account. You want to do it with Robinhood, do it. They give you the features to do that too. That's level one. Level two is just a very hard gate, but level two is decentralized finance, which means you buy the token. "I have a couple friends now in the space one of them is 23 yea